# Hierarchical Indexing

Up to this moment we are focusin on one-dimesional and two dimensional data, stored in Panda Series and DataFrame object, respectively. Often it is useful to go beyound this and store in higher-dimensional data- this is, data indexing by more then one or two keys. Common pattern is to make use of _hierachical indexing_ (multi indexing) to incorporate multi indexing levels whitin a single index. In this way higher-dimensional data can be compactly represented within the familiar one-dimesional Series and two-dimesional DafaFrame object.



In [12]:
import numpy as np
import pandas as pd

## A Multiply Index Series

Let's start by considering how we might represent two-dimensional data within a one-dimensional Series. For concreteness, we will consider a series of data where each point has a character and numerical key.

### The Bad way

Suppose you would like to trak data about states from two different year. Useing the Pandas tools, we've already cover, you might be tempted to simple use Pandas tuple as key:



In [13]:
index = [('California', 2000), ('California', 2010),
('New York', 2000), ('New York', 2010),
('Texas', 2000), ('Texas', 2010)]

populations = [33871648, 37253956,
18976457, 19378102,
20851820, 25145561]

pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

with this indexing scheme, you can straightforward index or slice the series based on this multiple index:

In [14]:
pop[("California", 2000):("Texas", 2000)]

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

But the convenience end there. For example if you need to select all values from 2010, you'll need to do some messy staff.

In [15]:
pop[[i for i in pop.index if i[1]==2010]] # select rows, whit index[1]==2010 

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### The Better way

Fortunately, Panda povides a better way. Our tuple-based indexing is essentually a rudimentary multi-index,and Pandas MultiIndex tyle given us the type of operations we wish to have. We create multi-index from the tuple a follows:

In [17]:
index=pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

Notice that the MultiIndex containa multi level of indexing- in the case the state names and the years, as well as multiple labels for each data point which encode these levels.

If we reindex our series with MultiIndex, we see the hierachical representation of the data.

In [18]:
pop=pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Here the fist columns of the Series representation show the multiple index values, while third column shows the data. Notice that some entries are missing in the firse column: it this multi.index representaiton, any blank entry indicates the same values as the line above it.

Now to access all data for which the second index in 2010, we simply use Panda slicing notation:

In [19]:
pop[:,2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

The result is a singly indexed array with just the keys we're interested in. The syntax is musch more convenient than the homespun tuple-based multi-indexing solution that we start with. 

### MultiIndex as extra dimesion

You might notice that we could easly have sored the same data using a simply DataFrame with index and column lables. In fact, Pandas is built with this equivalence in mind. The __unstack()__ method will quickly convert multiply-indexing Series into a conventionally index DataFrame:

In [28]:
pop_df=pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [29]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Why would bother with hierarchical indexing at all? The reason is simple: just as we were able to use multi indexing to represent data of three or more dimensional data within a one-dimensional Series, we can also used it to represent data of three or more dimensional is a Series or DataFrame.
Each extra level in a multi-index represent an extra dimension od data; 
Concretly, we might want to add another column of demographic data for ech state at each year (say, population under 18); with a MultiIndex this is a as easy as adding another column to the DataFrame:

In [33]:
pop_df=pd.DataFrame({"total":pop,"under18":[9267089, 9284094,
                                            4687374, 4318033,
                                            5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In addition, all the ufunc and other functionality discussed work with hierarchical indices as well. Here we comput the fraction of people under 18 by year:

In [37]:
f_u18=pop_df["under18"]/pop_df["total"]
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


This allows as to easily and quickly manipulate adn explore even high-dimensional data.

### Methods of MultiIndex Creation

The more streinghforward way to constract a multiply indexed Series or DataFrme is to simply pass a list of two or more index arrays to the constructor. For examle:


In [42]:
df=pd.DataFrame(np.random.rand(4,2), index=[["a","a","b","b"],[1,1,2,2]],columns=["data1","data2"])
df

data1     data2
a 1  0.152192  0.992379
  1  0.470351  0.768763
b 2  0.310678  0.284082
  2  0.643939  0.721803

Similary if ypu pass a dictionary with appropriate tuple as keys, Panda will automatically recognize this and used a MultiIndex by default:

In [43]:
data = {('California', 2000): 33871648,
('California', 2010): 37253956,
('Texas', 2000): 20851820,
('Texas', 2010): 25145561,
('New York', 2000): 18976457,
('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit Multiindex constructors

For more flexibility in how the index is constucted, you can instead use the class methond constructors avaible in the __pd.MultiIndex__.For example, as we did before,
you can construct the MultiIndex from a __simple list of arrays__, giving the index values
within each level:

In [44]:
pd.MultiIndex.from_arrays([["a","a","b","b"],[1,1,2,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 0, 1, 1]])

You can construct it from a __list of tuples__, giving the multiple index values of each point:

In [45]:
pd.MultiIndex.from_tuples([("a",1),("a",2),("b",1),("b",2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

You can even construct it from __Cartesin(decart) product__ of single indices:


In [47]:
pd.MultiIndex.from_product([["a","b"],[1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

Similary you can constranct the MultiIndex directly using its intermal encoding by passing __levels__(a list of lists containing avaible index values for each level) and __labels__(codes in new python)(a list od lists that reference these labels)

In [49]:
pd.MultiIndex(levels=[["a","b"],[1,2]],
                         codes=[[0,0,1,1],[0,1,0,1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

You can pass any of these objects as the index argument when creating a Series or
DataFrame, or to the reindex method of an existing Series or DataFrame

### MultiIndex level name

Sometimes it is convenientto name the levels of MultiIndex. You can accomplish this by passing the __names__ argument to any ot the above MultiIndex constructors, or by setting the __names__ attribute of index after the fact:




In [50]:
pop.index.names=["state","year"]
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for column

In a DataFrame, the rows and columns are completely symmetric, and just as the rows can have multiple leves of indices, the columns can have multiply leves as well. Consider the following, which is mock-up of some(samewhat realistic) medical data:



In [53]:
#hierarchical indices nd columns

index=pd.MultiIndex.from_product([[2013,2014],[1,2]], names=["year","visit"])
columns=pd.MultiIndex.from_product([["Bob","Guido","Sue"],["HR","Temp"]],names=["subject", "type"])

In [59]:
#mosk some data
data=np.round(np.random.randn(4,6),1)
data
data[:,::2]*=10
data+=37

In [60]:
# create DataFame
healt_data=pd.DataFrame(data, index=index, columns=columns)
healt_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  37.1  27.0  39.4  46.0  35.5
     2      23.0  37.1  34.0  37.2  32.0  37.1
2014 1      30.0  37.2  37.0  37.0  32.0  36.8
     2      36.0  38.1  42.0  37.8  33.0  37.0

Here we see where the multi-indexing for both rows and columns can come in very
handy. This is fundamentally four-dimensional data, where the dimensions are the
subject, the measurement type, the year, and the visit number. With this in place we
can, for example, index the top-level column by the person’s name and get a full Data
Frame containing just that person’s information:

In [61]:
healt_data["Bob"]

type          HR  Temp
year visit            
2013 1      43.0  37.1
     2      23.0  37.1
2014 1      30.0  37.2
     2      36.0  38.1

For complicated records containing multiple labeled measurements across multiple
times for many subjects (people, countries, cities, etc.), use of hierarchical rows and
columns can be extremely convenient!

### Indexing and Slicing a MultiIndex

Indexing and slicing on a MultiIndex is designed to be intuitive, and it help if you think about the indices as added dimensions.

#### Multiply indexed Series


In [62]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can access single element by indexing with multiple terms:


In [63]:
pop["California",2000]

33871648

The MultiIndex also support _partial indexing_, or indexing just one of the levels in the index. The result is another Series, with lower-level indices maintained:

In [64]:
pop["California"]

year
2000    33871648
2010    37253956
dtype: int64

__Partial slicing__ is available as well, as long as the MultiIndex is sorted

In [65]:
pop.loc["California":"New York"]

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

With sorted indices, we can perform partial indexing on lower levels by passing an
empty slice in the first index:

In [66]:
pop[:,2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

Other types of indexing and selection work as well; for example, selection based on Boolean masks:

In [67]:
pop[pop>22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

Selection based on fancy indexing also works:

In [68]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Multiply indexed DataFrame

A multiply indexed DataFrame behaves in a similar manner.

In [69]:
healt_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  37.1  27.0  39.4  46.0  35.5
     2      23.0  37.1  34.0  37.2  32.0  37.1
2014 1      30.0  37.2  37.0  37.0  32.0  36.8
     2      36.0  38.1  42.0  37.8  33.0  37.0

Remember that columns are primary in a DataFrame, and the syntax used form multiply indexed Series applies to columns.
For example, we can recover Guido's heart rate data with a simple operaion: 

In [70]:
healt_data["Guido","HR"]

year  visit
2013  1        27.0
      2        34.0
2014  1        37.0
      2        42.0
Name: (Guido, HR), dtype: float64

Also, as with the single-index case, we can use the loc, iloc:

In [79]:
healt_data.iloc[:3,:2] 

subject      Bob      
type          HR  Temp
year visit            
2013 1      43.0  37.1
     2      23.0  37.1
2014 1      30.0  37.2

These indexers provide an array-like view of the underlying two-dimensional data,
but each individual index in loc or iloc can be passed a tuple of multiple indices. For
example:

In [87]:
healt_data.loc[:, ('Bob', 'HR')] # all rows, column Bob and subcoumn HR

year  visit
2013  1        43.0
      2        23.0
2014  1        30.0
      2        36.0
Name: (Bob, HR), dtype: float64

In [88]:
healt_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  37.1  27.0  39.4  46.0  35.5
     2      23.0  37.1  34.0  37.2  32.0  37.1
2014 1      30.0  37.2  37.0  37.0  32.0  36.8
     2      36.0  38.1  42.0  37.8  33.0  37.0

Working with __slices__ within these index tuples is not especially convenient; trying to
create a slice within a tuple will lead to a syntax error:

In [82]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-82-fb34fa30ac09>, line 1)

You could get around this by building the desired slice explicitly using Python’s builtin
__slice()__ function, but a better way in this context is to use an __IndexSlice__ object,
which Pandas provides for precisely this situation. For example:

In [86]:
idx = pd.IndexSlice
healt_data.loc[idx[:, 1], idx[:, 'HR']]


,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,43.0,27.0,46.0
2014,1,30.0,37.0,32.0


### Rearranging Multi-Indices

One of the key to working with multiply indexed data is knowing how to effectively tranform the data. There a number of operation that will preserve all the information in the dataset, but rearrange it for the purposes of various computation.We
saw a brief example of this in the stack() and unstack() methods, but there are
many more ways to finely control the rearrangement of data between hierarchical
indices and columns, and we’ll explore them here.

#### Sorted and unsorted indices

Earlier, we briefly mentioned a caveatm but we should emphasize more here._Many of
the MultiIndex slicing operations will fail if the index is not sorted._ Let's take a look at this here.

we'll start by cretinf some simple multiple indexed data whre the indices are _not lexographically sort_. 



In [91]:
index=pd.MultiIndex.from_product([["a","c","b"],[1,2]])
data=pd.Series(np.random.rand(6), index=index)
data.index.names=["char","int"]
data

char  int
a     1      0.119131
      2      0.172100
c     1      0.350261
      2      0.020288
b     1      0.318070
      2      0.204330
dtype: float64

If we try to take a partial slice of this index, it will result in an error:

In [92]:
try:
    data["a":"b"]
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


For various reasons, partial slices and other similar operations
require the levels in the MultiIndex to be in sorted (i.e., lexographical) order.
Pandas provides a number of convenience routines to perform this type of sorting;
examples are the sort_index() and sortlevel() methods of the DataFrame. We’ll
use the simplest, sort_index(), here:

In [94]:
data=data.sort_index()
data

char  int
a     1      0.119131
      2      0.172100
b     1      0.318070
      2      0.204330
c     1      0.350261
      2      0.020288
dtype: float64

In [95]:
#With the index sorted in this way, partial slicing will work as expected:
data["a":"b"]

char  int
a     1      0.119131
      2      0.172100
b     1      0.318070
      2      0.204330
dtype: float64

#### Stackin and unstacking indices

As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, optionally specifying the level to use:

In [96]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [101]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [102]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


The opposite of __unstack()__ is __stack()__, which here can be used recover the original series:

In [103]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Index setting and resetting

Another way to rearrange hierarchical data is to turn the index labels into columns;
this can be accomplished with the __reset_index__ method. Calling this on the population
dictionary will result in a DataFrame with a state and year column holding the
information that was formerly in the index. For clarity, we can optionally specify the
name of the data for the column representation:




In [105]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [107]:
pop_flat=pop.reset_index(name="population")
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Often when you are working with data in the real world, the raw input data looks like
this and it’s useful to build a MultiIndex from the column values. This can be done
with the __set_index__ method of the DataFrame, which returns a multiply indexed Data
Frame:

In [108]:
pop_flat.set_index(["state","year"])


population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### Data Aggregations on Multi-Indices

We’ve previously seen that Pandas has built-in data aggregation methods, such as
mean(), sum(), and max(). For hierarchically indexed data, these can be passed a
level parameter that controls which subset of the data the aggregate is computed on.

In [109]:
healt_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  37.1  27.0  39.4  46.0  35.5
     2      23.0  37.1  34.0  37.2  32.0  37.1
2014 1      30.0  37.2  37.0  37.0  32.0  36.8
     2      36.0  38.1  42.0  37.8  33.0  37.0

Perhaps we’d like to average out the measurements in the two visits each year. We can
do this by naming the index level we’d like to explore, in this case the year:

In [114]:
data_mean = healt_data.mean(level='year') #level="vist"
data_mean # mean among rows!!

subject   Bob        Guido         Sue      
type       HR   Temp    HR  Temp    HR  Temp
year                                        
2013     33.0  37.10  30.5  38.3  39.0  36.3
2014     33.0  37.65  39.5  37.4  32.5  36.9

By further making use of the axis keyword, we can take the mean among levels on
the columns as well:

In [120]:
healt_data.mean(level='type',axis="columns")

type               HR       Temp
year visit                      
2013 1      38.666667  37.333333
     2      29.666667  37.133333
2014 1      33.000000  37.000000
     2      37.000000  37.633333

Thus in two lines, we’ve been able to find the average heart rate and temperature
measured among all subjects in all visits each year.